### 1.1 Toy model

Consider the following toy metabolic network with 8 reactions ($\mathrm{R}_1$, ..., $\mathrm{R}_8$) and 8 metabolites ($\mathrm{A}$, $\mathrm{B}$, $\mathrm{C}$, $\mathrm{D}$, $\mathrm{E}$, $\mathrm{ATP}$, $\mathrm{ADP}$, and $\mathrm{P}_\mathrm{i}$) (see supplementary pdf file).

(i) Specify which metabolites are transported across the cell boundaries (e.g., cell membrane), and the direction of transport.

Across the membrane:

All metabolites



(ii) Write down the stoichiometric matrix **S** using the ordering of reactions and metabolites as defined above. How many degrees of freedom does this reaction system have and what is the dimensionality of the solution space (i.e. null space of **S**)?

In [5]:
import numpy as np
from scipy.linalg import null_space
S = np.array([[1, 1, 0, 0, 1, 0, 0, 0],
              [0, 1, 1, 1, 0, 0, 0, 0],
              [0, 1, 1, 0, 0, 1, 0, 0],
              [0, 0, 0, 1, 1, 0, 0, 1],
              [0, 0, 0, 0, 1, 1, 0, 0],
              [0, 1, 1, 0, 1, 0, 1, 0],
              [0, 1, 1, 0, 1, 0, 1, 0],
              [0, 1, 1, 0, 1, 0, 1, 0]])

np.set_printoptions(formatter={'int': '{:3d}'.format})
print(S)
print("")
print("")

degrees_of_freedom = S.shape[1] - np.linalg.matrix_rank(S)
print(f"The array S has {degrees_of_freedom} degrees of freedom.")
print("")


null_space_S = null_space(S)
print("The null space of S is:")
print(null_space_S)


[[  1   1   0   0   1   0   0   0]
 [  0   1   1   1   0   0   0   0]
 [  0   1   1   0   0   1   0   0]
 [  0   0   0   1   1   0   0   1]
 [  0   0   0   0   1   1   0   0]
 [  0   1   1   0   1   0   1   0]
 [  0   1   1   0   1   0   1   0]
 [  0   1   1   0   1   0   1   0]]


The array S has 2 degrees of freedom.

The null space of S is:
[[ 6.45833983e-01  1.65356921e-01]
 [-5.55977134e-01  1.55636627e-01]
 [ 4.66120286e-01 -4.76630175e-01]
 [ 8.98568485e-02  3.20993548e-01]
 [-8.98568485e-02 -3.20993548e-01]
 [ 8.98568485e-02  3.20993548e-01]
 [ 1.79713697e-01  6.41987096e-01]
 [ 1.52655666e-16 -3.19189120e-16]]


(iii) Given an upper flux bound for $\mathrm{R}_1$ of 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, what is the maximal attainable flux through reaction $\mathrm{R}_8$ and the corresponding flux distribution? What is the net production of ATP (i.e., the flux through $\mathrm{R}_7$)? Implement the model using `cobrapy` and verify your answer by selecting $\mathrm{R}_8$ as the objective and maximizing its flux.

The `objective` is an attribute of the Model object, while the lower and upper flux bounds of a reaction is given by the attributes `lower_bound` and `upper_bound` of the corresponding Reaction object, respectively.

In [3]:
from cobra import Model, Reaction, Metabolite

# Define the model
model = Model('Metabolic_Network')

# Define the metabolites
A = Metabolite('A', compartment='c')
ADP = Metabolite('ADP', compartment='c')
Pi = Metabolite('Pi', compartment='c')
B = Metabolite('B', compartment='c')
C = Metabolite('C', compartment='c')
ATP = Metabolite('ATP', compartment='c')
D = Metabolite('D', compartment='c')
E = Metabolite('E', compartment='c')

# Add reactions to the model
R1 = Reaction('R1')
R1.name = 'R1'
R1.lower_bound = 0
R1.upper_bound = 10  # Upper flux bound for R1 is 10 mmol
R1.add_metabolites({A: 1})

R2 = Reaction('R2')
R2.name = 'R2'
R2.add_metabolites({A: -1, ADP: -1, Pi: -1, B: 1, C: 1, ATP: 1})

R3 = Reaction('R3')
R3.name = 'R3'
R3.add_metabolites({C: -1, ATP: -1, B: 1, ADP: 1, Pi: 1})

R4 = Reaction('R4')
R4.name = 'R4'
R4.add_metabolites({B: -1, D: 1})

R5 = Reaction('R5')
R5.name = 'R5'
R5.add_metabolites({A: -1, D: -1, ADP: -1, Pi: -1, E: 1, ATP: 1})

R6 = Reaction('R6')
R6.name = 'R6'
R6.add_metabolites({E: -1, C: 1})

R7 = Reaction('R7')
R7.name = 'R7'
R7.add_metabolites({ATP: -1, ADP: 1, Pi: 1})

R8 = Reaction('R8')
R8.name = 'R8'
R8.add_metabolites({D: -1})

# Add the reactions to the model
model.add_reactions([R1, R2, R3, R4, R5, R6, R7, R8])

# Set the objective as R8
model.objective = 'R8'

# Perform FBA
solution = model.optimize()

# Get the fluxes for all reactions and specifically the net production of ATP through R7
fluxes = solution.fluxes
R8_flux = fluxes['R8']
R7_flux = fluxes['R7']

R8_flux, R7_flux, fluxes

ModuleNotFoundError: No module named 'httpcore._utils'

(iv) It has been shown that the maximization of ATP yield in certain instances is a realistic cellular objective. Given the same flux bound for $\mathrm{R}_1$ as in (iii), explain and discuss the maximal feasible net production of ATP (i.e., flux through reaction $\mathrm{R}_7$). Verify your answer using `cobrapy`.

In [2]:
# Set the objective as R7
model.objective = 'R7'

# Perform FBA
solution = model.optimize()

# Get the fluxes for all reactions and specifically the net production of ATP through R7
fluxes = solution.fluxes
R7_flux = fluxes['R7']
R8_flux = fluxes['R8']

R7_flux, R8_flux, fluxes


NameError: name 'R7_flux' is not defined

(v) Assume that the flux of reaction $\mathrm{R}_6$ is to be constrained to zero. You may implement this in the stoichiometric matrix by adding a new row to **S** where all column entries are zero except in column 6, which is 1. Explain why this will constrain the flux of reaction $\mathrm{R}_6$ to zero. What is the dimensionality of this new stoichiometric matrix?

In [6]:
import numpy as np

# Add a new row to S
new_row = np.zeros((1, S.shape[1]))
new_row[0, 5] = 1
S_new = np.vstack((S, new_row))

# Print the dimensionality of the new stoichiometric matrix
print(f"The dimensionality of the new stoichiometric matrix S_new is: {S_new.shape}")


The dimensionality of the new stoichiometric matrix S_new is: (9, 8)


### 1.2 *Escherichia coli* core model

(i) Download `ecoli_core_model` from Blackboard and read the model using the `read_sbml_model` function in `cobrapy`. Give a description of its content (i.e., number of reactions, metabolites, genes, etc.). Which metabolic subsystems are implemented in the model?

Hint: the subsystems can be found in the Model attribute `groups`.

In [8]:
from cobra.io import read_sbml_model

# Read the model from the SBML file
model = read_sbml_model('ecoli_core_model.xml')
# Give a description of the model
print(f"Model description: {model.description}")

# Print the implemented metabolic subsystems
print("Implemented metabolic subsystems:")
for group in model.groups:
    print(group)



ModuleNotFoundError: No module named 'httpcore._utils'

(ii) Simulate the optimal growth phenotype on aerobic, minimal glucose media, by setting the lower bound of glucose uptake ('EX_glc__D_e') to a biologically reasonable uptake rate of -18.5 mmol gDW<sup>-1</sup> h<sup>-1</sup> and the oxygen uptake EX_o2_e to -1000 mmol gDW<sup>-1</sup> h<sup>-1</sup> <sup>i</sup>. What is the maximal specific growth rate and what are the uptake fluxes of glucose, ammonia, oxygen, and inorganic phosphate in the optimal solution?

<sup>i</sup> Note that uptake reaction flux bounds by default are negative, which is due to how these traditionally are defined in constraint-based models of metabolism. A boundary metabolite X is taken up by the system using the following format for the exchange reaction X $\Longleftrightarrow$, where a positive and negative flux denotes secretion and uptake, respectively.

In [ ]:
# Set the lower bound of glucose uptake and oxygen uptake
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -18.5
model.reactions.get_by_id('EX_o2_e').lower_bound = -1000

# Perform FBA
solution = model.optimize()

# Get the maximal specific growth rate
max_growth_rate = solution.objective_value

# Get the uptake fluxes
glucose_uptake_flux = solution.fluxes['EX_glc__D_e']
ammonia_uptake_flux = solution.fluxes['EX_nh4_e']
oxygen_uptake_flux = solution.fluxes['EX_o2_e']
inorganic_phosphate_uptake_flux = solution.fluxes['EX_pi_e']

# Print the results
print(f"Maximal specific growth rate: {max_growth_rate} h^-1")
print(f"Glucose uptake flux: {glucose_uptake_flux} mmol g^-1 h^-1")
print(f"Ammonia uptake flux: {ammonia_uptake_flux} mmol g^-1 h^-1")
print(f"Oxygen uptake flux: {oxygen_uptake_flux} mmol g^-1 h^-1")
print(f"Inorganic phosphate uptake flux: {inorganic_phosphate_uptake_flux} mmol g^-1 h^-1")


(iii) What is the secretion profile of anaerobic growth on glucose? Compare with that of aerobic growth on glucose (ii) and provide a biochemical explanation for their differences.

(iv) Visualize the reaction fluxes of both the aerobic and anaerobic flux phenotypes using the *E. coli* core model pathway map (found [here](https://escher.github.io/#/app?map=e_coli_core.Core\%20metabolism&tool=Builder&model=e_coli_core)) by creating a dictionary of reaction ids and corresponding fluxes, then writing this to a json file. Import the data into Escher by clicking Data $\rightarrow$ Load reaction data, then select your json file. Describe and discuss the difference in flux distributions.

In [ ]:
import json

# Create a dictionary of reaction IDs and fluxes
reaction_fluxes = {}
for reaction in model.reactions:
    reaction_fluxes[reaction.id] = solution.fluxes[reaction.id]

# Write the dictionary to a JSON file
with open('reaction_fluxes.json', 'w') as file:
    json.dump(reaction_fluxes, file)


(v) Setting the maximal substrate uptake flux to 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, maximize growth using each of the carbon sources listed in Table 1 individually under both aerobic and anaerobic conditions.

### Table 1 
| Substrate | Exchange reaction ID |
| --- | --- |
| acetate | EX_ac_e |
| acetaldehyde | EX_acald_e |
| 2-oxoglutarate | EX_akg_e | 
| ethanol | EX_etoh_e |
| D-fructose | EX_fru_e |
| fumarate | EX_fum_e |
| D-glucose | EX_glc__D_e |
| L-glutamine | EX_gln_L_e |
| L-glutamate | EX_glu_L_e |
| D-lactate | EX_lac_D_e |
| L-malate | EX_mal_L_e |
| pyruvate | EX_pyr_e |
| succinate | EX_succ_e |

In [ ]:
from cobra import Model, Reaction
from cobra.io import read_sbml_model

# Read the model from the SBML file
model = read_sbml_model('ecoli_core_model.xml')

# Define the carbon sources and their corresponding exchange reaction IDs
carbon_sources = {
    'acetate': 'EX_ac_e',
    'acetaldehyde': 'EX_acald_e',
    '2-oxoglutarate': 'EX_akg_e',
    'ethanol': 'EX_etoh_e',
    'D-fructose': 'EX_fru_e',
    'fumarate': 'EX_fum_e',
    'D-glucose': 'EX_glc__D_e',
    'L-glutamine': 'EX_gln_L_e',
    'L-glutamate': 'EX_glu_L_e',
    'D-lactate': 'EX_lac_D_e',
    'L-malate': 'EX_mal_L_e',
    'pyruvate': 'EX_pyr_e',
    'succinate': 'EX_succ_e'
}

# Perform FBA for each carbon source under both aerobic and anaerobic conditions
for carbon_source, exchange_reaction_id in carbon_sources.items():
    # Set the lower bound of the carbon source uptake to 10 mmol gDW^-1 h^-1
    model.reactions.get_by_id(exchange_reaction_id).lower_bound = -10
    
    # Set the objective as the biomass reaction
    model.objective = 'Biomass_Ecoli_core_w_GAM'
    
    # Perform FBA under aerobic conditions
    solution_aerobic = model.optimize()
    max_growth_rate_aerobic = solution_aerobic.objective_value
    
    # Set the lower bound of the oxygen uptake to 0 for anaerobic conditions
    model.reactions.get_by_id('EX_o2_e').lower_bound = 0
    
    # Perform FBA under anaerobic conditions
    solution_anaerobic = model.optimize()
    max_growth_rate_anaerobic = solution_anaerobic.objective_value
    
    # Print the results
    print(f"Carbon source: {carbon_source}")
    print(f"Maximal growth rate (aerobic): {max_growth_rate_aerobic} h^-1")
    print(f"Maximal growth rate (anaerobic): {max_growth_rate_anaerobic} h^-1")
    print("")
